# Mahsa Massoud - 810098032
## Genetic Algorithms

The goal of this computer assignment is to get more familiar with the genetic algorithm for scheduling. This algorithm is used when we have a state explosion problem, and our space is very large. Now, we want to set an appropriate schedule for doctors in specific shifts in a hospital. 
For every shift, there exists a minimum and maximum number of necessary doctors and there are also restricts due to doctor's health. 
When a doctor has a night shift , she or he cannot have another shift on the morning and afternoon on the following day.
Moreover, there is limitation for more 3 consecutive night shifts and there should not be more than 3 nights for one doctor in the plan.
Also, there exists a restriction on the maximum number of shifts in a schedule for doctors. 

In [ ]:
import random
import time
from math import floor
import numpy as np


These weights are for changing cost of each restriction in schedule. For now, they are all set 1 because there is no priority from one to others .

In [ ]:
WEIGHT_1  =1
WEIGHT_2 =1
WEIGHT_3=1
WEIGHT_4 =1

In [ ]:
def listtostring(s):
    str1 = "" 
    for ele in s: 
        str1 += str(ele)  

    return str1 

def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    file.close()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    return [days, list(range(doctors)), maxCapacity, allShifts]


# Definition of Chromosome
I define a chromosome as a probable soultion for the schedule and randomly select a list from doctors.

 # Fitness and Selection
 For finding fitness of each chromosome, I used fitness function and calculated cost of every constrains that explianed before.Then I sort them in order to find the best for the next generation.
 ELITISM selection method is implemented here which is to select a specific percentage of current generation for the next one.This means I copy 16% of the best chromosomes to the next generation, the remain, 84%, used for crossover and mutation.

# Crossover
For the implementation of crossover, I use the order 1 method. In the crossover method, I choose two different points and then copy the values between these points to the offspring and then complete the chromosome. The crossover happens in 84% of the population. 

# Mutation
For mutation, the result of the crossover is used for this method. In mutation function, I choose a random number between 0 to crossOverPoints, and base on this number, I swap the mapping of the chromosome.

### What if we just use crossover?
If we use just crossover in the genetic algorithm, its very potential to stuck in the local extremum for a long time. So, the mutation is an important part of the genetic algorithm.

### Mutation is more effective or crossover? Which of them lead to more accuracy?
Crossover leads to making a new generation. It makes completely new solutions and leads to significant improvement in the fitness scores. But, as I said, using just crossover may lead to stuck in some situations. So, mutation helps to change the current answer a little to make a possibly better generation.

### Why chromosomes get stuck in some situations?
Because they stuck in the local extremum.

### How do we solve this problem?
I use mutation multiple times. I use a random number between 0 to crossOverPoints for swap the genes.

For finding the fitness value of each chromosome, I used fitness function. In this function, I calculate the deciphered text and then calculate how many words in the deciphered text is in the dictionary. The number of words is the fitness value for each chromosome. Then, I sort them in reverse mode because I want to choose the best of them for the next generation.

In [28]:

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.popSize = 300
        self.pc = 0.65
        self.pm = 0.4
        self.goal = False
        self.numShifts = 3*self.days
        self.chromosomes = self.generateInitialPopulation()

        self.crossOverPoints = 3 * self.days 
        self.elitismPercentage = 16
        
    def minMax(self , day , shift):
       
        return self.allShifts[day][shift]
    
    def generateInitialPopulation(self):
        chromosomes = []
        for i in range(self.popSize):
            tmp = []
            for day in range(1,len(self.allShifts)+1):
                for shift in range(len(self.allShifts[day-1])):
                    randomList = random.sample(range(0,self.doctors - 1),np.random.randint(0,self.doctors - 1))
                    chromosome = listtostring(randomList) 
                    tmp.append(chromosome)
            chromosomes.append(tmp)
        return chromosomes
        
    def crossOver(self, chromosome1, chromosome2): 
        child1=[]
        child2=[]
        prev_index = 0
        chromo = False
        for i in range(2,self.crossOverPoints+1):
            rand_num = random.random()
            if rand_num > self.pc:
                if(chromo == False):
                    child1 += (chromosome2[prev_index:i-1])
                    child2 += (chromosome1[prev_index:i-1])
                    prev_index = i-1
                    chromo = True
                else:
                    child1 += (chromosome1[prev_index:i-1])
                    child2 +=(chromosome2[prev_index:i-1])
                    prev_index = i-1
                    chromo = False
        
        if(chromo == False):
            child1 += (chromosome2[prev_index:])
            child2 += (chromosome1[prev_index:])
        else:
            child1 += (chromosome1[prev_index:])
            child2 +=(chromosome2[prev_index:])
            
        return (child1,child2)
        
    def mutate(self, chromosome): 
        for i in range(1,self.crossOverPoints+1):
            generateRandom = random.uniform(0,1)
            if generateRandom < self.pm:
                randomList = random.sample(range(0,self.doctors - 1),len(chromosome[i-1])) 
                newNumber =  [str(x) for x in randomList]
                chromosome[i-1] = listtostring(newNumber)
        return chromosome
    

        
    def calculateFitness(self, chromosome):

        #capacity
        doctors = [0 for i in range(self.doctors)]
        validCapacities = 0
        unvalidcapacity = 0
        for i in chromosome:
            for j in i:
                doctors[int(j)] += 1
        for i in range(len(doctors)):
            if doctors[i] < self.maxCapacity+1:
                validCapacities += 1
            else :
                unvalidcapacity +=1
       
        # 3 night shifts
        keys = [i for i in range(self.doctors)]
        

        validDocs ={key: 1 for key in keys}
        unvaliddocs =0
        threeConsecutiveNight = 0
        for i in range(5,len(chromosome)+3,3):
            doctors = []
            for j in range(1,len(chromosome[i-3])+1):
                doctors.append(chromosome[i-3][j-1])
            if i + 3 < len(chromosome):
                for j in range(1,len(doctors)+1):
                    if doctors[j-1] in chromosome[i] and doctors[j-1] in chromosome[i + 3]:
                        validDocs[int(doctors[j-1])] = False
        for i in validDocs:
            if validDocs[i]:
                threeConsecutiveNight += 1
            else:
                unvaliddocs+=1
                
       #doctors night shift
        threeNights = 0
        doctors_days = [True for i in range(self.doctors)]
        for i in range(2,len(chromosome)-3,3):
            for j in range(1,len(chromosome[i])+1):
                if doctors_days[int(chromosome[i][j-1])] != 0:
                    if (chromosome[i][j-1] in chromosome[i+1]) or (chromosome[i][j-1] in chromosome[i+2]):
                        doctors_days[int(j-1)] = False
        counter = 0
        for i in doctors_days:
            if i == True:
                counter += 1
            else :
                threeNights +=1
                doctorOnNightShiftOk = 0
       
        unvalidshifts = 0
        doctorOnNightShiftOk = 0

        for day in range(1,len(self.allShifts)+1):
            for shift in range(1,len(self.allShifts[day-1])+1):
                t = self.minMax(day-1,shift-1)
                min_doctors = t[0]
                max_doctors = t[1]
                doctors_in_shift = len(chromosome[(day-1)*3 + (shift-1)])
                if  doctors_in_shift in range(t[0],t[1]+1) :
                    doctorOnNightShiftOk += 1
                else:
                    unvalidshifts +=1
        endFlag = True if (((validCapacities + threeConsecutiveNight + counter)/3) == self.doctors) else False
        if endFlag and doctorOnNightShiftOk == self.numShifts : 
            self.goal = True
            self.result = chromosome
        return  WEIGHT_1 *threeNights + WEIGHT_2*unvalidcapacity +WEIGHT_3 * unvaliddocs + WEIGHT_4 * unvalidshifts
    
    def generateNewPopulation(self):
        fitness = []
        for i in range(1,len(self.chromosomes)+1):
            fitness.append((self.calculateFitness(self.chromosomes[i-1]) ))
        self.chromosomes = [ x for _,x in sorted(zip(fitness,self.chromosomes) )]
        initialGeneration = self.chromosomes[:floor(self.elitismPercentage*0.01 * self.popSize)][:]
        for num in range(len(self.chromosomes)):
            p1 = np.random.randint(len(self.chromosomes))
            p2 = np.random.randint(len(self.chromosomes))
            if p1 == p2:
                p2 = np.random.randint(len(self.chromosomes))
            x = self.crossOver(self.chromosomes[p1],self.chromosomes[p2])
            self.chromosomes[p1] = x[0]
            self.chromosomes[p2] = x[1]
            index = np.random.randint(len(self.chromosomes))
            self.chromosomes[index] = self.mutate(self.chromosomes[index])
            
        return initialGeneration
    
    def schedule(self,name):
        while True:
            if(self.goal == True):
                break
            initialGeneration = self.generateNewPopulation()
            self.chromosomes[:floor(self.elitismPercentage*0.01 * self.popSize)] = initialGeneration[:]
        self.showResult(name)
        return

    def showResult(self,name):
        strr = ""
        for i in range (0,len(self.result), 3):
            s = []
            for j in range (i,i+3):
                s.append(self.result[j])
            for j in range(len(s)):
                if (s[j] == ''):
                    strr += "empty "
                elif len(s[j]) > 0:
                    for k in range(len(s[j])):
                        strr+= s[j][k]
                        strr += ","
                    strr = strr[:-1]
                    strr+= " "
            strr = strr[:-1]
            strr+= '\n'
        strr = strr[:-1]
        file = open(name,"w")
        file.write(strr)
        file.close()


In [31]:
for i in range(1,3):
    st = 'test'+str(i) + '.txt'
    fileInfo = readInput(st)
    name = 'output' + str(i) + '.txt'
    avg = []
    for j in range(2):
        tic = time.time()
        scheduler = JobScheduler(fileInfo)
        scheduler.schedule(name)
        toc = time.time()
        avg.append(toc-tic)
    print("average time for test",i,"is :" ,sum(avg) / len(avg) , "seconds")



average time for test 1 is : 4.598829627037048 seconds
average time for test 2 is : 17.906663060188293 seconds
